<a href="https://colab.research.google.com/github/sarabert96/Colexification/blob/main/02_DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cleaning Data

In [1]:
import pandas
import numpy
import nltk
nltk.download('punkt')
from collections import Counter
from google.colab import files

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Download original dataframe from Drive

In [2]:
!gdown --id 1O4Ofd0xX6I1eGcYPOQ0ET2_w-MEB5ybR #df_all_raw

Downloading...
From: https://drive.google.com/uc?id=1O4Ofd0xX6I1eGcYPOQ0ET2_w-MEB5ybR
To: /content/df_all_raw.csv
212MB [00:01, 116MB/s] 


Import libraries and data

In [3]:
df = pandas.read_csv('df_all_raw.csv')
df.head(2)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,dataset_ID,Form_ID,Form,clics_form,gloss_in_source,Concepticon_ID,Concepticon_Gloss,Ontological_Category,Semantic_Field,variety,Glottocode,ISO639P3code,Macroarea,Family,Latitude,Longitude
0,abrahammonpa,BugunBichom-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Bichom,bich1234,NaN,NaN,Sino-Tibetan,NaN,NaN
1,abrahammonpa,BugunKaspi-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Kaspi,kasp1234,NaN,NaN,Sino-Tibetan,NaN,NaN


Remove diachronic varieties (middle, old, classic, ancient)

In [4]:
#tokenize variety --> new column
df['tk_variety']=df['variety'].apply(nltk.tokenize.word_tokenize)

In [5]:
df['flag']=0 # add flag column

In [6]:
# flag non wanted varieties
diachronic_varieties = ['Old', 'Middle', 'Classical', 'Ancient', 'English','Latin', 'Proto', 'Gothic']
for i in range(len(df['tk_variety'])):
  r = df['tk_variety'].iloc[i]
  if any((v in diachronic_varieties) for v in r):
    df['flag'].iloc[i]+=1


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [7]:
# delete flagged columns with non wanted categories
df= df.drop(df[df.flag > 0].index)

In [8]:
df.shape

(1350177, 18)

Now we have the original dataframe removed from unwanted varieties

Remove also multiword concepticon: containing OR and/or parenthesis

In [9]:
# modify column with word count: flag > 0 if multiword
df['flag'] = df['Concepticon_Gloss'].str.count(' OR ')
# delete flagged columns
df = df.drop(df[df.flag > 0].index)
df['flag'] = df['Concepticon_Gloss'].str.count("\(") # \ symbol is for regex problem
# delete flagged columns
df = df.drop(df[df.flag > 0].index)

In [10]:
# delete flag column
del df['flag']

In [11]:
df.shape

(1202826, 17)

df is now without bad concepts (OR, bracket)

Check for null values

In [12]:
df['Concepticon_Gloss'].isna().values.any()

False

In [13]:
# reset index
df = df.reset_index(drop=True)

To lowercase

In [18]:
df['Concepticon_Gloss'] = df['Concepticon_Gloss'].str.lower()

In [19]:
df.head(1)

,dataset_ID,Form_ID,Form,clics_form,gloss_in_source,Concepticon_ID,Concepticon_Gloss,Ontological_Category,Semantic_Field,variety,Glottocode,ISO639P3code,Macroarea,Family,Latitude,Longitude,tk_variety
0,abrahammonpa,BugunBichom-100_gold-1,san,san,gold,1369,gold,Person/Thing,Basic actions and technology,Bugun Bichom,bich1234,NaN,NaN,Sino-Tibetan,NaN,NaN,"[Bugun, Bichom]"


Save df on csv

In [20]:
df.to_csv('df_all_ok.csv')

Remember to download the csv

In [21]:
files.download('df_all_ok.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>